In [27]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [28]:
#ee.Authenticate()
#geemap.update_package()

ee.Initialize()

Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)

ga_pt = ee.Feature(ee.Geometry.Point([-81.422318, 31.539096]))

ga_pts = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point([-81.422318, 31.539096]), {'site_id': '1'}),
  ee.Feature(ee.Geometry.Point([-81.295564, 31.535967]), {'site_id': '2'}),
  ee.Feature(ee.Geometry.Point([-81.229141, 31.518847]), {'site_id': '3'}),
  ee.Feature(ee.Geometry.Point([-81.365805, 31.451791]), {'site_id': '4'}),
  ee.Feature(ee.Geometry.Point([-81.340722, 31.435814]), {'site_id': '5'}),
  ee.Feature(ee.Geometry.Point([-81.280011, 31.387962]), {'site_id': '6'}),
  ee.Feature(ee.Geometry.Point([-81.478498, 31.334813]), {'site_id': '7'}),
  ee.Feature(ee.Geometry.Point([-81.415535, 31.30824]), {'site_id': '8'}),
  ee.Feature(ee.Geometry.Point([-81.333707, 31.351518]), {'site_id': '9'}),
  ee.Feature(ee.Geometry.Point([-81.270863, 31.476948]), {'site_id': '10'})
])

##Adding every plot coordinate
allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots_NEW.csv'
fc_all = geemap.csv_to_ee(allplots_fc, latitude = "Latitude", longitude = "Longitude")

# Map

In [29]:
##FLATS
def addFLATS(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B6'),
            'RED': image.select('B4'),
            'GREEN': image.select('B3')
        })
    
    return image.addBands(flats.rename('flats'))

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B5'),
            'RED': image.select('B3'),
            'GREEN': image.select('B2')
        })
    
    return image.addBands(flats.rename('flats'))

In [30]:
##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.
def maskL5sr(image):
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5

  qa = image.select('pixel_qa')

  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  return image.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"])

In [37]:
#Coincident imagery
l8_col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filter(ee.Filter.calendarRange(1998, 2023,'year')) \
    .filterBounds(ga_pts)

l7_col = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filter(ee.Filter.calendarRange(1998, 2023,'year')) \
    .filterBounds(ga_pts)

l5_col = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filter(ee.Filter.calendarRange(1998, 2023,'year')) \
    .filterBounds(ga_pts)

l9_col = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2") \
    .filter(ee.Filter.calendarRange(1998, 2023,'year')) \
    .filterBounds(ga_pts)
##ALL MIGRATED TO COLLECTION 2!

In [38]:
print(l5_col.size().getInfo())
print(l7_col.size().getInfo())
print(l8_col.size().getInfo())
print(l9_col.size().getInfo())

505
905
569
55


In [39]:
#Filter dates to get coincident images between l5 and L7; L7 and L8; and L8 and L9
#https://gis.stackexchange.com/questions/386827/filter-two-image-collections-by-the-same-aqusition-date-in-google-earth-engine

def datefunc(image):
    date = ee.Date(image.get('system:time_start')).format("YYYY-MM-dd")
    date = ee.Date(date)
    return image.set('date', date)

x_8 = l8_col.map(datefunc)
x_7 = l7_col.map(datefunc)
x_5 = l5_col.map(datefunc)
x_9 = l9_col.map(datefunc)

filterTimeEq = ee.Filter.equals(
    leftField='date',
    rightField='date'
)

simpleJoin = ee.Join.simple()

#Shared_5/7
L57 = ee.ImageCollection(simpleJoin.apply(x_5, x_7, filterTimeEq))

#Shared_7/8
L78 = ee.ImageCollection(simpleJoin.apply(x_7, x_8, filterTimeEq))

#Shared_8/9
L89 = ee.ImageCollection(simpleJoin.apply(x_8, x_9, filterTimeEq))

#https://developers.google.com/earth-engine/guides/joins_simple

In [40]:
print('Landsat 5/7 coincident images: ' + str(L57.size().getInfo()))
print('Landsat 7/8 coincident images: ' + str(L78.size().getInfo()))
print('Landsat 8/9 coincident images: ' + str(L89.size().getInfo()))

L89.aggregate_array('system:id').getInfo()

#Only ONE instance of same-day imagery for all 3 sensors over the 20+-year period

Landsat 5/7 coincident images: 0
Landsat 7/8 coincident images: 3
Landsat 8/9 coincident images: 2


['LANDSAT/LC08/C02/T1_L2/LC08_016038_20211112',
 'LANDSAT/LC08/C02/T1_L2/LC08_016039_20211112']

In [46]:
#Near coincident imagery - within one day (or 86400000 milliseconds)

filterNear = ee.Filter.maxDifference(
    difference = 86400000,
    leftField='system:time_start',
    rightField='system:time_start'
)

N57 = ee.ImageCollection(simpleJoin.apply(l5_col, l7_col, filterNear))
N75 = ee.ImageCollection(simpleJoin.apply(l7_col, l5_col, filterNear))

N78 = ee.ImageCollection(simpleJoin.apply(l7_col, l8_col, filterNear))
N87 = ee.ImageCollection(simpleJoin.apply(l8_col, l7_col, filterNear))

N89 = ee.ImageCollection(simpleJoin.apply(l8_col, l9_col, filterNear))
N98 = ee.ImageCollection(simpleJoin.apply(l9_col, l8_col, filterNear))

print('Landsat 5/7 NEAR coincident images: ' + str(N57.size().getInfo()))
print('Landsat 7/8 NEAR coincident images: ' + str(N78.size().getInfo()))
print('Landsat 8/9 NEAR coincident images: ' + str(N89.size().getInfo()))

# print('Landsat 7/5 NEAR coincident images: ' + str(N75.size().getInfo()))
# print('Landsat 8/7 NEAR coincident images: ' + str(N87.size().getInfo()))
# print('Landsat 9/8 NEAR coincident images: ' + str(N98.size().getInfo()))

#Remember: this is just in Georgia! Discrepencies in image collection sizes due to some duplicate dates bewteen images
#See: https://gis.stackexchange.com/questions/345962/create-paired-images-by-near-date
#327, 257, 49

Landsat 5/7 NEAR coincident images: 328
Landsat 7/8 NEAR coincident images: 256
Landsat 8/9 NEAR coincident images: 52


In [44]:
# N57.aggregate_array('system:id').getInfo()
# N78.aggregate_array('system:id').getInfo()
N89.aggregate_array('system:id').getInfo()

['LANDSAT/LC08/C02/T1_L2/LC08_016038_20211112',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20211214',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20211230',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220115',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220131',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220216',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220304',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220320',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220421',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220507',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220523',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220608',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220624',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220710',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220726',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220811',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220827',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20220912',
 'LANDSAT/LC08/C02/T1_L2/LC08_016039_20211112',
 'LANDSAT/LC08/C02/T1_L2/LC08_016039_20211214',
 'LANDSAT/LC08/C02/T1_L2/LC08_016039_202

In [45]:
# N75.aggregate_array('system:id').getInfo()
# N87.aggregate_array('system:id').getInfo()
N98.aggregate_array('system:id').getInfo()

#Some duplicate dates

['LANDSAT/LC09/C02/T1_L2/LC09_016038_20211206',
 'LANDSAT/LC09/C02/T1_L2/LC09_016038_20220107',
 'LANDSAT/LC09/C02/T1_L2/LC09_016038_20220224',
 'LANDSAT/LC09/C02/T1_L2/LC09_016038_20220328',
 'LANDSAT/LC09/C02/T1_L2/LC09_016038_20220413',
 'LANDSAT/LC09/C02/T1_L2/LC09_016038_20220429',
 'LANDSAT/LC09/C02/T1_L2/LC09_016038_20220515',
 'LANDSAT/LC09/C02/T1_L2/LC09_016038_20220531',
 'LANDSAT/LC09/C02/T1_L2/LC09_016038_20220616',
 'LANDSAT/LC09/C02/T1_L2/LC09_016038_20220702',
 'LANDSAT/LC09/C02/T1_L2/LC09_016038_20220718',
 'LANDSAT/LC09/C02/T1_L2/LC09_016038_20220803',
 'LANDSAT/LC09/C02/T1_L2/LC09_016038_20220819',
 'LANDSAT/LC09/C02/T1_L2/LC09_016038_20220904',
 'LANDSAT/LC09/C02/T1_L2/LC09_016038_20220920',
 'LANDSAT/LC09/C02/T1_L2/LC09_016039_20211206',
 'LANDSAT/LC09/C02/T1_L2/LC09_016039_20220107',
 'LANDSAT/LC09/C02/T1_L2/LC09_016039_20220224',
 'LANDSAT/LC09/C02/T1_L2/LC09_016039_20220328',
 'LANDSAT/LC09/C02/T1_L2/LC09_016039_20220413',
 'LANDSAT/LC09/C02/T1_L2/LC09_016039_202

In [36]:
# l7_col.aggregate_array('system:id').getInfo()
# l5_col.aggregate_array('system:id').getInfo()
# l8_col.aggregate_array('system:id').getInfo()
# l9_col.aggregate_array('system:id').getInfo()

In [47]:
x = ee.Image('LANDSAT/LE07/C02/T1_L2/LE07_016038_20130623')
props = geemap.image_props(x)
# props.getInfo()

In [33]:
# l7_col.aggregate_array('system:id').getInfo()
# l5_col.aggregate_array('system:id').getInfo()
# l8_col.aggregate_array('system:id').getInfo()
# l9_col.aggregate_array('system:id').getInfo()

In [ ]:
#Mask clouds from images, add FLATS   
    
#     .map(maskL5sr) \
#     .map(addFLATS) \

In [106]:
#Tide determination: finding dates that are similar (%?) on near coincident images
tides = pd.read_csv('C:/Users/arj26323/Documents/Data/flats/water_level_filtered_2013_2022.csv')
tides['Date'] = pd.to_datetime(tides['Date'])
display(tides)
z = tides.iloc[:,[1,4,5]]
z

dictide = tides.groupby('Date')['Verified_m'].agg(list).to_dict()
dictide

##group by day and get av height? Or just around 15:54?

Unnamed: 0       Date  Predicted_m Preliminary_m Verified_m   time
0               1 2013-01-01        0.826             -      0.854  15:54
1               2 2013-01-01        0.809             -      0.842  16:00
2               3 2013-01-01        0.789             -      0.831  16:06
3               4 2013-01-02        0.843             -      0.811  15:54
4               5 2013-01-02        0.841             -      0.816  16:00
...           ...        ...          ...           ...        ...    ...
10681       10682 2022-09-29        1.125             -      1.662  16:00
10682       10683 2022-09-29        1.104             -      1.662  16:06
10683       10684 2022-09-30        1.215             -      1.362  15:54
10684       10685 2022-09-30        1.212             -      1.392  16:00
10685       10686 2022-09-30        1.208             -        1.4  16:06

[10686 rows x 6 columns]

{Timestamp('2013-01-01 00:00:00'): ['0.854', '0.842', '0.831'],
 Timestamp('2013-01-02 00:00:00'): ['0.811', '0.816', '0.818'],
 Timestamp('2013-01-03 00:00:00'): ['0.946', '0.934', '0.962'],
 Timestamp('2013-01-04 00:00:00'): ['0.641', '0.701', '0.731'],
 Timestamp('2013-01-05 00:00:00'): ['0.098', '0.146', '0.193'],
 Timestamp('2013-01-06 00:00:00'): ['-0.476', '-0.425', '-0.371'],
 Timestamp('2013-01-07 00:00:00'): ['-0.721', '-0.688', '-0.659'],
 Timestamp('2013-01-08 00:00:00'): ['-0.978', '-0.993', '-0.984'],
 Timestamp('2013-01-09 00:00:00'): ['-0.974', '-1.012', '-1.047'],
 Timestamp('2013-01-10 00:00:00'): ['-0.569', '-0.622', '-0.674'],
 Timestamp('2013-01-11 00:00:00'): ['-0.041', '0.021', '-0.126'],
 Timestamp('2013-01-12 00:00:00'): ['0.416', '0.359', '0.299'],
 Timestamp('2013-01-13 00:00:00'): ['0.873', '0.817', '0.77'],
 Timestamp('2013-01-14 00:00:00'): ['1.13', '1.084', '1.11'],
 Timestamp('2013-01-15 00:00:00'): ['1.138', '1.138', '1.126'],
 Timestamp('2013-01-16 00:

In [92]:
y57 = pd.DataFrame(N57.aggregate_array('LANDSAT_PRODUCT_ID').getInfo(),columns = ['Image'], index = None)
y57['Sensor'] = 'Landsat 5/7'

y75 = pd.DataFrame(N75.aggregate_array('LANDSAT_PRODUCT_ID').getInfo(),columns = ['Image'], index = None)
y75['Sensor'] = 'Landsat 7/5'

y78 = pd.DataFrame(N78.aggregate_array('LANDSAT_PRODUCT_ID').getInfo(),columns = ['Image'], index = None)
y78['Sensor'] = 'Landsat 7/8'

y87 = pd.DataFrame(N87.aggregate_array('LANDSAT_PRODUCT_ID').getInfo(),columns = ['Image'], index = None)
y87['Sensor'] = 'Landsat 8/7'

y89 = pd.DataFrame(N89.aggregate_array('LANDSAT_PRODUCT_ID').getInfo(),columns = ['Image'], index = None)
y89['Sensor'] = 'Landsat 8/9'

y98 = pd.DataFrame(N98.aggregate_array('LANDSAT_PRODUCT_ID').getInfo(),columns = ['Image'], index = None)
y98['Sensor'] = 'Landsat 9/8'

df = pd.concat([y57,y75,y78,y87,y89,y98], axis = 0)
df['Date'] = pd.to_datetime(df.Image.str.split('_').str[3])
df

Image       Sensor       Date
0   LT05_L2SP_016038_19990711_20200907_02_T1  Landsat 5/7 1999-07-11
1   LT05_L2SP_016038_19990727_20200907_02_T1  Landsat 5/7 1999-07-27
2   LT05_L2SP_016038_19990812_20200907_02_T1  Landsat 5/7 1999-08-12
3   LT05_L2SP_016038_19990828_20200907_02_T1  Landsat 5/7 1999-08-28
4   LT05_L2SP_016038_19990913_20200907_02_T1  Landsat 5/7 1999-09-13
..                                       ...          ...        ...
44  LC09_L2SP_017038_20220709_20220711_02_T1  Landsat 9/8 2022-07-09
45  LC09_L2SP_017038_20220725_20220727_02_T1  Landsat 9/8 2022-07-25
46  LC09_L2SP_017038_20220810_20220812_02_T1  Landsat 9/8 2022-08-10
47  LC09_L2SP_017038_20220826_20220830_02_T1  Landsat 9/8 2022-08-26
48  LC09_L2SP_017038_20220911_20220913_02_T1  Landsat 9/8 2022-09-11

[1360 rows x 3 columns]

In [110]:
# df1 = pd.merge(df,z, on = 'Date', how = 'left') #Lots of duplicate dates, it seems
# df1

df['Tide_height'] = df['Date'].map(dictide) 
df

Image       Sensor       Date  \
0   LT05_L2SP_016038_19990711_20200907_02_T1  Landsat 5/7 1999-07-11   
1   LT05_L2SP_016038_19990727_20200907_02_T1  Landsat 5/7 1999-07-27   
2   LT05_L2SP_016038_19990812_20200907_02_T1  Landsat 5/7 1999-08-12   
3   LT05_L2SP_016038_19990828_20200907_02_T1  Landsat 5/7 1999-08-28   
4   LT05_L2SP_016038_19990913_20200907_02_T1  Landsat 5/7 1999-09-13   
..                                       ...          ...        ...   
44  LC09_L2SP_017038_20220709_20220711_02_T1  Landsat 9/8 2022-07-09   
45  LC09_L2SP_017038_20220725_20220727_02_T1  Landsat 9/8 2022-07-25   
46  LC09_L2SP_017038_20220810_20220812_02_T1  Landsat 9/8 2022-08-10   
47  LC09_L2SP_017038_20220826_20220830_02_T1  Landsat 9/8 2022-08-26   
48  LC09_L2SP_017038_20220911_20220913_02_T1  Landsat 9/8 2022-09-11   

                 Tide_height  
0                        NaN  
1                        NaN  
2                        NaN  
3                        NaN  
4                        NaN  
..                       ...  
44  [-0.987, -0.932, -0.916]  
45  [-1.017, -1.038, -1.046]  
46   [-1.267, -1.313, -1.36]  
47  [-0.212, -0.263, -0.306]  
48     [0.677, 0.619, 0.568]  

[1360 rows x 4 columns]